In [1]:
!python -V

Python 3.9.12


In [2]:
import pandas as pd

/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [3]:
pd.__version__

'2.2.3'

In [8]:
import pickle

In [9]:
import seaborn as sns
import matplotlib.pyplot as plt

In [10]:

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
# from sklearn.metrics import root_mean_squared_error

In [11]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-training/02-experiment-tracking/mlruns/1', creation_time=1748289679069, experiment_id='1', last_update_time=1748289679069, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [12]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [13]:
df1 = pd.read_parquet('./data/green_tripdata_2023-01.parquet')

In [14]:
df1.head(5)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.9,1.0,0.5,4.03,0.0,None,1.0,24.18,1.0,1.0,2.75
1,2,2023-01-01 00:51:03,2023-01-01 00:57:49,N,1.0,24,43,1.0,1.81,10.7,1.0,0.5,2.64,0.0,None,1.0,15.84,1.0,1.0,0.00
2,2,2023-01-01 00:35:12,2023-01-01 00:41:32,N,1.0,223,179,1.0,0.00,7.2,1.0,0.5,1.94,0.0,None,1.0,11.64,1.0,1.0,0.00
3,1,2023-01-01 00:13:14,2023-01-01 00:19:03,N,1.0,41,238,1.0,1.30,6.5,0.5,1.5,1.70,0.0,None,1.0,10.20,1.0,1.0,0.00
4,1,2023-01-01 00:33:04,2023-01-01 00:39:02,N,1.0,41,74,1.0,1.10,6.0,0.5,1.5,0.00,0.0,None,1.0,8.00,1.0,1.0,0.00


In [15]:
df_train = read_dataframe('./data/green_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2023-02.parquet')

In [16]:
len(df_train), len(df_val)

(65946, 62574)

In [17]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [18]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [19]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [21]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

6.033417590654431

In [22]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [23]:
with mlflow.start_run():

    mlflow.set_tag("developer", "uma")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2023-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2023-01.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [24]:
import xgboost as xgb

In [25]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [26]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [27]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [28]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|                                                              | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:02:48] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.52772                                                                               
[1]	validation-rmse:5.59433                                                                               
[2]	validation-rmse:5.31575                                                                               
[3]	validation-rmse:5.23179                                                                               
[4]	validation-rmse:5.19945                                                                               
[5]	validation-rmse:5.18649                                                                               
[6]	validation-rmse:5.17959                                                                               
[7]	validation-rmse:5.17696                                                                               
[8]	validation-rmse:5.17926                                                                               
[9]	validation-rmse:5.17989          

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:03:15] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.22048                                                                               
[1]	validation-rmse:6.20073                                                                               
[2]	validation-rmse:5.69560                                                                               
[3]	validation-rmse:5.47845                                                                               
[4]	validation-rmse:5.35844                                                                               
[5]	validation-rmse:5.31626                                                                               
[6]	validation-rmse:5.29338                                                                               
[7]	validation-rmse:5.28591                                                                               
[8]	validation-rmse:5.27976                                                                               
[9]	validation-rmse:5.27703          

...

[0]	validation-rmse:8.85153                                                                               
[1]	validation-rmse:8.43002                                                                               
[2]	validation-rmse:8.05381                                                                               
[3]	validation-rmse:7.71908                                                                               
[4]	validation-rmse:7.42195                                                                               
[5]	validation-rmse:7.15914                                                                               
[6]	validation-rmse:6.92737                                                                               
[7]	validation-rmse:6.72358                                                                               
[8]	validation-rmse:6.54425                                                                               
[9]	validation-rmse:6.38703          

[76]	validation-rmse:5.25727                                                                              
[77]	validation-rmse:5.25723                                                                              
[78]	validation-rmse:5.25700                                                                              
[79]	validation-rmse:5.25500                                                                              
[80]	validation-rmse:5.25491                                                                              
[81]	validation-rmse:5.25470                                                                              
[82]	validation-rmse:5.25311                                                                              
[83]	validation-rmse:5.25285                                                                              
[84]	validation-rmse:5.25255                                                                              
[85]	validation-rmse:5.25087         

[152]	validation-rmse:5.23071                                                                             
[153]	validation-rmse:5.23065                                                                             
[154]	validation-rmse:5.23030                                                                             
[155]	validation-rmse:5.23013                                                                             
[156]	validation-rmse:5.23026                                                                             
[157]	validation-rmse:5.23001                                                                             
[158]	validation-rmse:5.22986                                                                             
[159]	validation-rmse:5.22991                                                                             
[160]	validation-rmse:5.22965                                                                             
[161]	validation-rmse:5.22969        

[228]	validation-rmse:5.22354                                                                             
[229]	validation-rmse:5.22357                                                                             
[230]	validation-rmse:5.22346                                                                             
[231]	validation-rmse:5.22338                                                                             
[232]	validation-rmse:5.22331                                                                             
[233]	validation-rmse:5.22313                                                                             
[234]	validation-rmse:5.22310                                                                             
[235]	validation-rmse:5.22298                                                                             
[236]	validation-rmse:5.22305                                                                             
[237]	validation-rmse:5.22304        

KeyboardInterrupt: 

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)